<a href="https://colab.research.google.com/github/rohitsubodh/Rohit/blob/main/evalution_mode_collapse_dcgan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:

from os import makedirs
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy.random import randn
from numpy.random import randint
from keras.datasets.mnist import load_data
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose,Dropout
from keras.layers import LeakyReLU
from keras.layers import BatchNormalization
from keras.initializers import RandomNormal
from matplotlib import pyplot

In [20]:
# define the standalone discriminator model
def define_discriminator(in_shape=(28,28,1)):
    model = Sequential()
    init = RandomNormal(stddev=0.02)
    model.add(Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init,input_shape=in_shape))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.2))
    # model.add(Dropout(0.4))
    model.add(Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init,input_shape=in_shape))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.2))
    # model.add(Dropout(0.4))
    
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
# compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model


In [21]:
# define the standalone generator model
def define_generator(latent_dim):
    model=Sequential()
    init = RandomNormal(stddev=0.02)
    n_nodes = 128 * 7 * 7
    model.add(Dense(n_nodes, kernel_initializer=init, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=.2))
    model.add(Reshape((7,7,128)))
    # upsample to 14x14
    model.add(Conv2DTranspose(128,(4,4),strides=(2,2),padding='same',kernel_initializer=init))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=.2))
    # upsample to 14x14
    model.add(Conv2DTranspose(128,(4,4),strides=(2,2),padding='same',kernel_initializer=init))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=.2))
    
    model.add(Conv2D(1,(7,7),activation='sigmoid',padding='same',kernel_initializer=init))
    return model

In [22]:
# define the combined generator and discriminator model, for updating the generator
def define_gan(g_model, d_model):
# make weights in the discriminator not trainable
   d_model.trainable = False
# connect them
   model = Sequential()
# add generator
   model.add(g_model)
# add the discriminator
   model.add(d_model)
# compile model
   opt = Adam(lr=0.0002, beta_1=0.5)
   model.compile(loss='binary_crossentropy', optimizer=opt)
   return model


def load_real_samples():
# load mnist dataset
    (trainX, trainy), (_, _) = load_data()
# expand to 3d, e.g. add channels
    X = expand_dims(trainX, axis=-1)
# select all of the examples for a given class
    selected_ix = trainy == 8
    X = X[selected_ix]
# convert from ints to floats
    X = X.astype('float32')
# scale from [0,255] to [-1,1]
    X = (X - 127.5) / 127.5
    return X


In [23]:
def generate_real_samples(dataset, n_samples):
# choose random instances
       ix = randint(0, dataset.shape[0], n_samples)
# retrieve selected images
       X = dataset[ix]
# generate ✬real✬ class labels (1)
       y = ones((n_samples, 1))
       return X, y

def generate_latent_points(latent_dim, n_samples):
# generate points in the latent space
     x_input = randn(latent_dim * n_samples)
# reshape into a batch of inputs for the network
     x_input = x_input.reshape(n_samples, latent_dim)
     return x_input        

# use the generator to generate n fake examples, with class labels
def generate_fake_samples(g_model, latent_dim, n_samples):
# generate points in latent space
     x_input = generate_latent_points(latent_dim, n_samples)
# predict outputs
     X = g_model.predict(x_input)
# create ✬fake✬ class labels (0)
     y = zeros((n_samples, 1))
     return X, y
     

In [24]:


# generate samples and save as a plot and save the model
def summarize_performance(step, g_model, latent_dim, n_samples=100):
# prepare fake examples
    X, _ = generate_fake_samples(g_model, latent_dim, n_samples)
# scale from [-1,1] to [0,1]
    X = (X + 1) / 2.0
    for i in range(10 * 10):
# define subplot
        pyplot.subplot(10, 10, 1 + i)
# turn off axis
        pyplot.axis('off')
# plot raw pixel data
        pyplot.imshow(X[i, :, :, 0], cmap='gray_r')
# save plot to file
    pyplot.savefig('results_collapse/generated_plot_%03d.png' % (step+1))
    pyplot.close()
# save the generator model
    g_model.save('results_collapse/model_%03d.h5' % (step+1))

# create a line plot of loss for the gan and save to file
def plot_history(d1_hist, d2_hist, g_hist, a1_hist, a2_hist):
# plot loss
     pyplot.subplot(2, 1, 1)
     pyplot.plot(d1_hist, label='d-real')
     pyplot.plot(d2_hist, label='d-fake')
     pyplot.plot(g_hist, label='gen')
     pyplot.legend()
# plot discriminator accuracy
     pyplot.subplot(2, 1, 2)
     pyplot.plot(a1_hist, label='acc-real')
     pyplot.plot(a2_hist, label='acc-fake')
     pyplot.legend()
# save plot to file
     pyplot.savefig('results_collapse/plot_line_plot_loss.png')
     pyplot.close()
    


In [25]:
# train the generator and discriminator
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=10, n_batch=128):
     # calculate the number of batches per epoch
       bat_per_epo = int(dataset.shape[0] / n_batch)
# calculate the total iterations based on batch and epoch
       n_steps = bat_per_epo * n_epochs
# calculate the number of samples in half a batch
       half_batch = int(n_batch / 2)
# prepare lists for storing stats each iteration
       d1_hist, d2_hist, g_hist, a1_hist, a2_hist = list(), list(), list(), list(), list()
# manually enumerate epochs
       for i in range(n_steps):
# get randomly selected ✬real✬ samples
           X_real, y_real = generate_real_samples(dataset, half_batch)
# update discriminator model weights
           d_loss1, d_acc1 = d_model.train_on_batch(X_real, y_real)
# generate ✬fake✬ examples
           X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
# update discriminator model weights
           d_loss2, d_acc2 = d_model.train_on_batch(X_fake, y_fake)
# prepare points in latent space as input for the generator
           X_gan = generate_latent_points(latent_dim, n_batch)
# create inverted labels for the fake samples
           y_gan = ones((n_batch, 1))
# update the generator via the discriminator✬s error
           g_loss = gan_model.train_on_batch(X_gan, y_gan)
# summarize loss on this batch
           print('>%d, d1=%.3f, d2=%.3f g=%.3f, a1=%d, a2=%d' %(i+1, d_loss1, d_loss2, g_loss, int(100*d_acc1), int(100*d_acc2)))
# record history
           d1_hist.append(d_loss1)
           d2_hist.append(d_loss2)
           g_hist.append(g_loss)
           a1_hist.append(d_acc1)
           a2_hist.append(d_acc2)
# evaluate the model performance every ✬epoch✬
           if (i+1) % bat_per_epo == 0:
                  summarize_performance(i, g_model, latent_dim)
       plot_history(d1_hist, d2_hist, g_hist, a1_hist, a2_hist)


In [26]:
# make folder for results
makedirs('results_collapse', exist_ok=True)
# size of the latent space
latent_dim = 1
# create the discriminator
d_model = define_discriminator()
# create the generator7.7. Complete Example of GAN for MNIST 123
g_model = define_generator(latent_dim)
# create the gan
gan_model = define_gan(g_model, d_model)
# load image data
dataset = load_real_samples()
print(dataset.shape)
# train model
train(g_model, d_model, gan_model, dataset, latent_dim)


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


(5851, 28, 28, 1)
>1, d1=1.023, d2=0.397 g=0.681, a1=26, a2=100
>2, d1=0.238, d2=0.075 g=0.667, a1=96, a2=100
>3, d1=0.111, d2=0.030 g=0.653, a1=98, a2=100
>4, d1=0.064, d2=0.019 g=0.644, a1=100, a2=100
>5, d1=0.045, d2=0.014 g=0.631, a1=100, a2=100
>6, d1=0.048, d2=0.011 g=0.625, a1=100, a2=100
>7, d1=0.036, d2=0.009 g=0.616, a1=100, a2=100
>8, d1=0.026, d2=0.008 g=0.611, a1=100, a2=100
>9, d1=0.019, d2=0.007 g=0.600, a1=100, a2=100
>10, d1=0.016, d2=0.006 g=0.597, a1=100, a2=100
>11, d1=0.016, d2=0.005 g=0.587, a1=100, a2=100
>12, d1=0.013, d2=0.005 g=0.577, a1=100, a2=100
>13, d1=0.013, d2=0.004 g=0.575, a1=100, a2=100
>14, d1=0.012, d2=0.004 g=0.570, a1=100, a2=100
>15, d1=0.010, d2=0.004 g=0.562, a1=100, a2=100
>16, d1=0.010, d2=0.003 g=0.554, a1=100, a2=100
>17, d1=0.008, d2=0.003 g=0.550, a1=100, a2=100
>18, d1=0.009, d2=0.003 g=0.546, a1=100, a2=100
>19, d1=0.009, d2=0.003 g=0.539, a1=100, a2=100
>20, d1=0.007, d2=0.003 g=0.531, a1=100, a2=100
>21, d1=0.005, d2=0.002 g=0.525, a